In [11]:
from datetime import datetime   
from My_functions import *

In [12]:
df = pd.read_excel("Link_facebook.xlsx")

In [13]:
with open(r'D:\craw data facebook\facebook\facebook_credentials.txt') as file:
    EMAIL = file.readline().split('"')[1]
    PASSWORD = file.readline().split('"')[1]

In [14]:
def _login(browser, email, password):
    browser.get('https://www.facebook.com/login')
    email_input = browser.find_element("id", "email")
    password_input = browser.find_element("id", "pass")
    email_input.send_keys(email)
    password_input.send_keys(password)
    login_button = browser.find_element("name", "login")
    login_button.click()
    time.sleep(random.randint(5, 10))

In [15]:
# Cấu hình các tùy chọn cho Chrome
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--dns-prefetch-disable")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-gpu")

# Khởi tạo ChromeDriver với các tùy chọn đã thiết lập
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

current_day = datetime.now().strftime('%m-%d-%Y')

# Đăng nhập vào Facebook
_login(driver, EMAIL, PASSWORD)

# Mở file để ghi dữ liệu
with open('output_Data.txt', 'a', encoding='utf-8') as f:
    # Duyệt qua từng Fanpage hoặc Group
    for i in range(0, len(df)):
        try:
            data_page = {
                "Hệ Kênh": df['Hệ Kênh'][i], 
                "Tên page": df['Tên Page'][i],
                "Link page": df['Link'][i],
                "Số like": None,
                "Số Follower": None,
                "Số Người tham gia": None, 
                "Nền tảng": df['Nền tảng'][i], 
                "Ngày save": current_day
            }

            # Truy cập vào trang
            driver.get(data_page['Link page'])
            time.sleep(random.randint(3, 6))  # Đợi để trang tải hoàn tất

            # Lấy mã nguồn HTML của trang
            page_source = driver.page_source

            # Sử dụng BeautifulSoup để phân tích HTML
            soup = BeautifulSoup(page_source, 'html.parser')
            scroll_until_end(driver, duration=random.randint(1, 3))

            # Trường hợp là Group, lấy số người tham gia
            member_element = soup.find('a', href=lambda href: href and '/members/' in href)
            if member_element:
                members_text = member_element.get_text().strip()
                members_number = members_text.split()[0]  # Giả sử số người tham gia là phần đầu tiên
                data_page['Số Người tham gia'] = members_number

            # Lấy số lượt thích
            like_element = soup.find('a', href=lambda href: href and '/friends_likes/' in href)
            if like_element:
                likes = like_element.get_text().split()[0]  # Lấy phần text đầu tiên, thường là số likes
                data_page['Số like'] = likes

            # Lấy số lượng followers
            follower_element = soup.find('a', href=lambda href: href and '/followers/' in href)
            if follower_element:
                followers_text = follower_element.get_text().strip()
                followers_number = followers_text.split()[0]  # Lấy số đầu tiên
                data_page['Số Follower'] = followers_number

            # Ghi dữ liệu ngay lập tức vào file sau mỗi lần lấy xong dữ liệu từ một trang
            f.write(f"Hệ Kênh: {data_page['Hệ Kênh']}\n")
            f.write(f"Tên page: {data_page['Tên page']}\n")
            f.write(f"Link page: {data_page['Link page']}\n")
            f.write(f"Số like: {data_page['Số like']}\n")
            f.write(f"Số Follower: {data_page['Số Follower']}\n")
            f.write(f"Số Người tham gia: {data_page['Số Người tham gia']}\n")
            f.write(f"Nền tảng: {data_page['Nền tảng']}\n")
            f.write(f"Ngày save: {data_page['Ngày save']}\n")  # Ghi ngày lưu dữ liệu
            f.write("-" * 40 + "\n")

            # Nếu cả 3 trường đều là None, ghi vào file lỗi
            if data_page['Số like'] is None and data_page['Số Follower'] is None and data_page['Số Người tham gia'] is None:
                with open('link_loi.txt', 'a', encoding='utf-8') as error_file:
                    error_file.write(f"Hệ Kênh: {data_page['Hệ Kênh']}\n")
                    error_file.write(f"Tên page: {data_page['Tên page']}\n")
                    error_file.write(f"Link page: {data_page['Link page']}\n")
                    error_file.write(f"Nền tảng: {data_page['Nền tảng']}\n")
                    error_file.write("-" * 40 + "\n")

        except Exception as e:
            # Nếu có lỗi ngoại lệ, ghi thông tin về link lỗi vào file
            with open('link_loi.txt', 'a', encoding='utf-8') as error_file:
                error_file.write(f"Hệ Kênh: {data_page['Hệ Kênh']}\n")
                error_file.write(f"Tên page: {data_page['Tên page']}\n")
                error_file.write(f"Link page: {data_page['Link page']}\n")
                error_file.write(f"Nền tảng: {data_page['Nền tảng']}\n")
                error_file.write("-" * 40 + "\n")

            # Tiếp tục với liên kết tiếp theo mà không dừng chương trình
            continue


In [21]:
data_all_facebook = convert_to_dataframe_from_txt("output_Data.txt")


In [22]:
data_page = data_all_facebook[data_all_facebook['Nền tảng'] == "Fanpage"]

In [23]:
data_group = data_all_facebook[data_all_facebook["Nền tảng"] == "Group"]

In [24]:
save_file(data_page,"page")

'File đã được lưu với tên: D:\\BeatVn\\Data\\Data_10-22-2024\\page_facebook_10-22-2024.xlsx'

In [25]:
save_file(data_group,"group")

'File đã được lưu với tên: D:\\BeatVn\\Data\\Data_10-22-2024\\group_facebook_10-22-2024.xlsx'